Hi, I'm SalvadOR, responsible for creating a school timetable to organize classes, teachers, and rooms for an upcoming semester.

We have some strict requirements to meet. This is a very hard problem we face every year, and I need your help to design an optimal timetable.

Here's the situation:

There are 4 classes, each requiring specific teaching sessions.
There are 4 teachers, and each teacher has assigned subjects to teach.
There are 4 rooms, and only one class can occupy a room during any given period.
The timetable spans 30 periods, and we must ensure that all requirements are met without any conflicts or overlaps.
I desperately need a timetable that satisfies all requirements (each class meets with the right teacher in the right room the required number of times), avoiding any type of clashes such as double-booking a teacher, room, or class during the same period.

We say a timetable is optimized when it minimizes idle periods and maximizes resource utilization (teachers and rooms).

Can you help me solve this problem?

## Solution modeled with binary variables

**Goal**

Minimizes idle periods and maximize resource utilization. However, this approach may yield scattered solutions.

$$ \max \sum_{c,t,r,p} a_{ctrp} $$

**Decisions**:

$$a_{ctrp}\in\{0,1\} \text{ assigns room r with teacher t and class c at period }p$$


**Data:**


Qualified combinations required for scheduling $Q = \{(c,t,r)\} \subset C\times T\times R$

Demand of sessions each class $c$ with each teacher $t$ needs, given for each room $r$ by $d_{ctr}$

Set of classes $C=\{1..4\}$

Set of teachers $T=\{1..4\}$

Set of rooms $R=\{1..4\}$

Set of periods  $P=\{1..30\}$

Set of subjects $S=\{1..20\}$

**Constraints**

| **Description** | **Expression** |
| --- | --- |
| Meet the session demand  | $\forall (c, t, r) \in Q: \sum_p x_{ctrp} = d_{ctr}$ |
| No class clash | $\forall c, p: \sum_{t,r} x_{ctrp}\leq1$ |
| No teacher clash | $\forall t, p: \sum_{r,c}a_{ctrp}\leq1$ |
| No room clash | $\forall r, p: \sum_{t,c}a_{ctrp}\leq1$ |

In [13]:
def get_instance(input_filename):
    with open(input_filename, "r") as file:
        data = file.read()

    # Split the data into lines
    lines = data.strip().split('\n')
    l = 0

    # Skip comment lines
    while lines[l].startswith('#') or lines[l] == '':
        l += 1
    
    # get metadata parameters (always first line)
    metadata = map(int, lines[l].split())
    data = []
    l += 1

    # Process each line
    while l < len(lines):
        data.append(list(map(int, lines[l].split())))
        l += 1

    return *metadata, data

def parse_to_zero_index(data, indexes):
    for i in range(len(data)):
        for idx in indexes:
            data[i][idx] -= 1
    return

In [14]:
def correct_format(data):
    for i in range(100):
        for _ in range(7):
            data[i].extend(data[i+1])
            data.pop(i+1)
    return

In [16]:
num_classes, num_subjects, num_teachers, num_rooms, max_periods, num_req, requirements = get_instance("data/classroom.txt")
parse_to_zero_index(requirements, [0, 1, 2, 3])

# Print the extracted data
print(f"Number of classes: {num_classes}")
print(f"Number of subjects: {num_subjects}")
print(f"Number of teachers: {num_teachers}")
print(f"Number of rooms: {num_rooms}")
print(f"Max of periods: {max_periods}")
print(f"Requirements matrix: {len(requirements)}")

Number of classes: 4
Number of subjects: 20
Number of teachers: 4
Number of rooms: 4
Max of periods: 35
Requirements matrix: 120


In [17]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()
solver = cp_model.CpSolver()

In [18]:
x = {}

for c in range(num_classes):
    for t in range(num_teachers):
        for r in range(num_rooms):
            for p in range(max_periods):
                x[c,t,r,p] = model.NewBoolVar(f'schedule_class_{c}_by_{t}_in_{r}_at_{p}')

In [19]:
# 1. Each requirement must be satisfied the exact number of times.
for c, _, t, r, d in requirements:
    model.Add(sum(x[c,t,r,p] for p in range(max_periods)) == d )

In [20]:
# 2. A class can only be assigned to one teacher-room-period at a time.
for c in range(num_classes):
    for p in range(max_periods):
        model.Add(sum(x[c,t,r,p] for t in range(num_teachers) for r in range(num_rooms)) <= 1)

In [21]:
# 3. A teacher can only teach one class in one room at a time.
for t in range(num_teachers):
    for p in range(max_periods):
        model.Add(sum(x[c,t,r,p] for c in range(num_classes) for r in range(num_rooms)) <= 1)

In [22]:
# 4. A room can only host one class at a time.
for r in range(num_rooms):
    for p in range(max_periods):
        model.Add(sum(x[c,t,r,p] for c in range(num_classes) for t in range(num_teachers)) <= 1)

In [23]:
# Specify the type of problem. In this case, we want to minimize the objective function
solver.parameters.num_search_workers = 8
solver.parameters.max_time_in_seconds = 120
model.Minimize(
    sum(x[c,t,r,p] 
        for c in range(num_classes) 
        for t in range(num_teachers)
        for r in range(num_rooms)
        for p in range(max_periods)
    )
)

In [24]:
# Call the solver method to find the optimal solution
callback = cp_model.ObjectiveSolutionPrinter()
or_status = solver.SolveWithSolutionCallback(model, callback)
status = solver.StatusName(or_status)

if status in ["OPTIMAL", "FEASIBLE"]:
    print(f'Solution: Total cost of worker\'s payment = {solver.ObjectiveValue()}')
else:
    print('A solution could not be found, check the problem specification')

Solution 0, time = 0.05 s, objective = 59
Solution: Total cost of worker's payment = 59.0


In [87]:
solution1 = set()
for t in range(num_tasks):
    for w in range(num_workers):
        if solver.Value(x[t][w]) == 1:
            solution1.add((t, w))
# [solver.Value(x[s][v]) for v in range(num_cities) if x[s][v] is not None]

In [88]:
sum(assigment_costs[t][w] for t, w in solution1)

305

Greedy Approach

In [101]:
from collections import deque

pq = deque(
    sorted((assigment_costs[t][w], t, w) 
           for t in range(num_tasks) 
           for w in range(num_workers)
        )
    )

solution2 = set()
chosen_tasks = set()
chosen_workers = set()
total_cost = 0

while len(solution2) < 100:
    cost, t, w = pq.popleft()
    if t in chosen_tasks or w in chosen_workers:
        continue
    
    solution2.add((t, w))
    chosen_tasks.add(t)
    chosen_workers.add(w)
    total_cost += cost
    

In [ ]:
sum(assigment_costs[t][w] for t, w in solution2)

522

: 

Greedy doesn't work. Example

| | W1 | W2 |
|--|---|---|
|T1 | 1 | 2 |
|T2 | 3 | 100|